In [ ]:
print("hello oshin")

In [ ]:
! pip install jiwer

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
file_path = "/kaggle/input/personal-data/Translator Dataset/Sentence.xlsx"

In [ ]:
train_data = pd.read_excel(file_path)

print(train_data.shape)
train_data.head(3)

In [ ]:
train_df = train_data[['Sylhet Language', 'Standard Bangla Lanuguage']]

print(train_df.shape)

train_df.head()


In [ ]:
train_df.isna().sum()

In [ ]:
 train_df = train_df.rename(columns={'Sylhet Language': 'blang'})
 train_df = train_df.rename(columns={'Standard Bangla Lanuguage': 'tlang'})

train_df.sample()

In [ ]:
def calculate_sentence_length(sentence):
    return len(sentence)

# Apply the function to the column and create a new column for sentence length
train_df['sl1'] = train_df['blang'].apply(calculate_sentence_length)
train_df['sl2'] = train_df['tlang'].apply(calculate_sentence_length)

train_df.head(3)

In [ ]:
train_df.sl2.describe()

In [ ]:
df_sorted = train_df.sort_values(by='sl1', ascending=True)  # Use ascending=False for descending order

# Reset the index if needed
df_sorted = df_sorted.reset_index(drop=True)


df_sorted.head(10)

In [ ]:
df_sorted = df_sorted[3:]

df_sorted.head(3)

In [ ]:
pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5", use_fast=False)

In [ ]:
input_sentence = "আমি তোমাকে ভালবাসি।"

input_sentence = df_sorted['tlang'][10]

print(f"Input sentence = {input_sentence}")
input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids
generated_tokens = model.generate(input_ids, max_length = 128)
decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]

print(decoded_tokens)

In [ ]:
tokenizer.tokenize(input_sentence)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.15, shuffle=True, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(train_df.shape, val_df.shape)

In [ ]:
train_df.head(3)

In [ ]:
val_df.head(3)

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_pandas(train_df)
ds_eval = Dataset.from_pandas(val_df)

In [ ]:
ds_train, ds_eval

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "microsoft/phi-2"

# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

# model_checkpoint = "google/umt5-base"
model_checkpoint='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
model.cuda()

In [ ]:
def prepare_dataset(sample):
    output = tokenizer(sample["blang"])
    output["labels"] = tokenizer(sample["tlang"])['input_ids']
    output["length"] = len(output["labels"])
    return output


ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
ds_eval = ds_eval.map(prepare_dataset, remove_columns=ds_eval.column_names)

# ds_train_dv  = ds_train_dv.map(prepare_dataset, remove_columns=ds_train_dv.column_names)
# ds_eval_dv = ds_eval_dv.map(prepare_dataset, remove_columns=ds_eval_dv.column_names)

# ds_train_merged = ds_train_merged.map(prepare_dataset, remove_columns=ds_train_merged.column_names)
# ds_eval_merged = ds_eval_merged.map(prepare_dataset, remove_columns=ds_eval_merged.column_names)

In [ ]:
print(ds_train)
print(ds_eval)

# print('dataverse: ')
# print(ds_train_dv)
# print(ds_eval_dv)

# print('merged: ')
# print(ds_train_merged)
# print(ds_eval_merged)

In [ ]:
import numpy as np
from datasets import load_metric

wer_metric = load_metric("wer")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": result}

In [ ]:
Epochs = 50
batch_size = 16
Epochs

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    group_by_length=True,
    length_column_name="length",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    metric_for_best_model="wer",
    greater_is_better=False,
    load_best_model_at_end=True,
    num_train_epochs=Epochs,
    save_steps=2000,  # Save every 200 steps
    eval_steps=50,  # Evaluate every 200 steps
    logging_steps=50,  # Log every 50 steps
    learning_rate=3e-4,
    weight_decay=1e-2,
    warmup_steps=100,  # Adjusted for smaller dataset
    save_total_limit=1,
    predict_with_generate=True,
    generation_max_length=128,
    push_to_hub=False,
    report_to="none",
)


In [ ]:
from transformers import EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./kaggle/working/trained_model/")

<a href="/kaggle/working/file.zip"> Download File </a>

In [ ]:
# !rm -rf /kaggle/working/

In [ ]:
val_df.head(3)

In [ ]:
text = val_df['blang'][0]
text, val_df['tlang'][0]

In [ ]:
inputs = tokenizer(text, return_tensors='pt')
inputs = inputs['input_ids'].to('cuda')
outputs = model.generate(inputs, max_length=128)
print(tokenizer.decode(outputs[0], skip_special=True))

In [ ]:
outputs

In [ ]:
import numpy as np
ri = np.random.randint(0, 10)
print(ri)
text = val_df['blang'][ri]
text, val_df['tlang'][ri]

In [ ]:
def sek(text):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = inputs['input_ids'].to('cuda')
    outputs = model.generate(inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special=True)

In [ ]:
import numpy as np
ri = np.random.randint(0, 10)
print(ri)
text = val_df['blang'][ri]
print('Test: ',text, end=' ')
print('True: ',val_df['tlang'][ri])

sek(text)

In [ ]:
# inputs = tokenizer('বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশনাল ডাইরেক্টরগেমস অ্যান্ড স্পোর্টস ডিপার্টমেন্ট', return_tensors='pt')
# inputs = inputs['input_ids'].to('cuda')
# outputs = model.generate(inputs, max_length=512)
# print(tokenizer.decode(outputs[0], skip_special=True))